In [4]:
import numpy as np
import torch
import os
from torch.utils.data import DataLoader, SubsetRandomSampler
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
import time
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
import torch.optim as optim
from torchvision import models
import torch.nn as nn

In [5]:
import os
# NewPlantDiease/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train
path = '/home/NewPlantDiease/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)'
train_data1 = path + '/train'
val_data1 = path + '/valid'
diease = os.listdir(train_data1)

In [7]:
mean = [0.4757, 0.5001, 0.4264]
std = [0.2165, 0.1957, 0.2320]

transforms = transform.Compose([transform.Resize((256,256)), transform.ToTensor(), transform.Normalize(mean = mean, std = std)])
val_datasets = ImageFolder(root = train_data1, transform=transforms)
train_data1 = int(0.2 * len(val_datasets))
testing_data1 = len(val_datasets) - train_data1
train_set1, val_set1 = torch.utils.data.random_split(val_datasets, [train_data1, testing_data1])
kfold = KFold(n_splits=10)
avg_train_acc = []
avg_test_acc = []
avg_train_loss = []
avg_test_loss = []
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_set1)):
    print('*********************',fold)
    train_sampler = SubsetRandomSampler(train_ids)
    
    test_sampler = SubsetRandomSampler(test_ids)
    
    Train_dataloader = DataLoader(train_set1, batch_size = 512, sampler = train_sampler)
    Val_dataloader = DataLoader(train_set1, batch_size = 512, sampler = test_sampler)
len(train_ids),len(Train_dataloader),len(test_sampler), len(train_sampler)

********************* 0
********************* 1
********************* 2
********************* 3
********************* 4
********************* 5
********************* 6
********************* 7
********************* 8
********************* 9


(12654, 25, 1405, 12654)

In [6]:
class EarlyStopping:
    def __init__(self, patience):
        self.pateince  = patience
        self.best_score = None
        self.counter = 0
        self.stop  = False
    def _stopping(self,val_loss):
        if self.best_score is None:
            self.best_score = val_loss
        elif val_loss > self.best_score:
            self.counter += 1
            if self.counter == self.pateince:
                self.stop = True
        else:
            self.best_score = val_loss
            # print(self.best_score)
            counter = 0

In [11]:
def resnet():
    from torchvision import models
    import torch.nn as nn
    model = models.resnet18(pretrained=True)
    # model
    # Freeze all parameters in the pre-trained model
    for param in model.parameters():
        param.requires_grad = True
    # # Extracting features from last CNN layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features , 38)
    return model
    

In [12]:
def training(epoch,Train_dataloader,val_dataloader):
    from torchvision import models
    import torch.nn as nn
    model = models.resnet18(pretrained=True)
    # model
    # Freeze all parameters in the pre-trained model
    for param in model.parameters():
        param.requires_grad = True
    # # Extracting features from last CNN layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features , 38)
    # import torch.optim as optim
    optimizer = optim.Adam(model.parameters(), lr = 1e-5)
    loss_function = nn.CrossEntropyLoss()
    earlystopping = EarlyStopping(patience=5)
    epoch = epoch
    Train_dataloader = Train_dataloader
    Val_dataloader = val_dataloader
    for epoch in range(epoch):
        start = time.time()
    
        tr_acc = 0
        val_acc = 0
#         avg_train_acc = []
#         avg_test_acc = []
#         avg_train_loss = []
#         avg_test_loss = []
    
    # Train
        model.train()
    
        with tqdm(Train_dataloader, unit="batch") as tepoch:
            for xtrain, ytrain in tepoch:
            
                train_prob = model(xtrain)
        
            
                train_loss = loss_function(train_prob, ytrain)
                train_loss.backward() # backpropagation

            # loss.backward()
                optimizer.step() # update the model parameters
                optimizer.zero_grad()
            # training ends
            
                train_pred = torch.max(train_prob, 1).indices
                tr_acc += int(torch.sum(train_pred == ytrain))
            
            ep_tr_acc = tr_acc / len(Train_dataloader)
        end = time.time()
        duration = (end - start) / 60
        
    # print(f"Epoch: {epoch}, Test_Loss: {train_loss}, Train_acc: {ep_tr_acc}")
    # Evaluate
    
        model.eval()
#     # with torch.no_grad():
        with torch.no_grad():
            for xval, yval in Val_dataloader:
                val_prob = model(xval)
            
                val_loss = loss_function(val_prob , yval)
            
                val_pred = torch.max(val_prob,1).indices
                val_acc += int(torch.sum(val_pred == yval))
            
            ep_val_acc = val_acc / len(Val_dataloader)
        avg_val_loss = val_loss / len(Val_dataloader)
        earlystopping._stopping(val_loss)
        if earlystopping.stop:
            print('Early Stopping Excuated')
            break
        end = time.time()
        duration = (end - start) / 60
    
        print(f"Epoch: {epoch}, Test_Loss: {train_loss},Validation_loss: {val_loss}, Train_acc: {ep_tr_acc}, Val_acc: {ep_val_acc}")
        
    avg_train_acc.append(ep_tr_acc)
    avg_test_acc.append(ep_val_acc)
    avg_train_loss.append(train_loss.detach().item())
    avg_test_loss.append(float(val_loss.item()))
    print(f'Fold{fold}Append list is printing')
    print(avg_train_acc,avg_test_acc,avg_test_loss,avg_train_loss)
    
    

In [13]:
def get_normalizationvalue(Train_dataloader):
    import numpy as np
    s1 = torch.tensor([0.0, 0.0, 0.0])
    s2 = torch.tensor([0.0, 0.0, 0.0])
    for image,_ in tqdm(Train_dataloader):
    # print(image.size())
        s1 += image.sum(axis=[0,2,3])
        s2 += (image**2).sum(axis=[0,2,3])
    n = LEN_TRAIN * image.size(2) * image.size(3)

    mean = s1 / n
    std = np.sqrt((s2/n) - ((s1/n)**2))
    return mean, std

In [14]:
def data_loader(path):
    mean = [0.4757, 0.5001, 0.4264]
    std = [0.2165, 0.1957, 0.2320]
    transforms = transform.Compose([transform.Resize((256,256)), transform.ToTensor(), transform.Normalize(mean = mean, std = std)])
    dataset = ImageFolder(root = path, transform = transforms)
    return dataset
    # StratifiedKFold(n_splits=split)
                                   

In [15]:
def model():
    val_datasets = data_loader(val_data1)
    train_data1 = int(0.95 * len(val_datasets))
    testing_data1 = len(val_datasets) - train_data1
    train_set1, val_set1 = torch.utils.data.random_split(val_datasets, [train_data1, testing_data1])
    kfold = KFold(n_splits=3)
    avg_train_acc = []
    avg_test_acc = []
    avg_train_loss = []
    avg_test_loss = []
    for fold, (train_ids, test_ids) in enumerate(kfold.split(val_set1)):
        print('*********************',fold)
        train_sampler = SubsetRandomSampler(train_ids)
        test_sampler = SubsetRandomSampler(test_ids)
        Train_dataloader = DataLoader(val_set1, batch_size = 256, sampler = train_sampler)
        Val_dataloader = DataLoader(val_set1, batch_size = 256, sampler = test_sampler)
        # model = resnet()
        training(3,Train_dataloader,Val_dataloader)
        
# model()

In [7]:
mean = [0.4757, 0.5001, 0.4264]
std = [0.2165, 0.1957, 0.2320]

transforms = transform.Compose([transform.Resize((256,256)), transform.ToTensor(), transform.Normalize(mean = mean, std = std)])
val_datasets = ImageFolder(root = train_data1, transform=transforms)
train_data1 = int(0.25 * len(val_datasets))
testing_data1 = len(val_datasets) - train_data1
train_set1, val_set1 = torch.utils.data.random_split(val_datasets, [train_data1, testing_data1])
kfold = KFold(n_splits=4)
avg_train_acc = []
avg_test_acc = []
avg_train_loss = []
avg_test_loss = []
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_set1)):
    print('*********************',fold)
    train_sampler = SubsetRandomSampler(train_ids)
    test_sampler = SubsetRandomSampler(test_ids)
    Train_dataloader = DataLoader(train_set1, batch_size = 512, sampler = train_sampler)
    Val_dataloader = DataLoader(train_set1, batch_size = 512, sampler = test_sampler)
    # Creating model
    model = models.resnet18(pretrained=True)
# model
# Freeze all parameters in the pre-trained model
    for param in model.parameters():
        param.requires_grad = True
# # Extracting features from last CNN layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features , 38)
    # optimizer and loss function
    # import torch.optim as optim
    optimizer = optim.Adam(model.parameters(), lr = 1e-2)
    loss_function = nn.CrossEntropyLoss()
    earlystopping = EarlyStopping(patience=5)
    for epoch in range(10):
        start = time.time()
    
        tr_acc = 0
        val_acc = 0

    
        # Train
        model.train()
        with tqdm(total=len(Train_dataloader)) as pbar:
        #with tqdm(Train_dataloader, unit="batch") as tepoch:
            for xtrain, ytrain in Train_dataloader:
            
                train_prob = model(xtrain)
        
            
                train_loss = loss_function(train_prob, ytrain)
                train_loss.backward() # backpropagation

            # loss.backward()
                optimizer.step() # update the model parameters
                optimizer.zero_grad()
            # training ends
            
                train_pred = torch.max(train_prob, 1).indices
                tr_acc += int(torch.sum(train_pred == ytrain))
                
                pbar.set_postfix(tr_acc = tr_acc, train_loss = train_loss)
                pbar.update(n=1)
            
            ep_tr_acc = tr_acc / len(train_ids)
        end = time.time()
        duration = (end - start) / 60
    
    # print(f"Epoch: {epoch}, Test_Loss: {train_loss}, Train_acc: {ep_tr_acc}")
    # Evaluate
    
        model.eval()
#     # with torch.no_grad():
        with torch.no_grad():
            #with...
            for xval, yval in Val_dataloader:
                val_prob = model(xval)
            
                val_loss = loss_function(val_prob , yval)
            
                val_pred = torch.max(val_prob,1).indices
                val_acc += int(torch.sum(val_pred == yval))
            
            ep_val_acc = val_acc / len(test_ids)
        # avg_val_loss = val_loss / len(Val_dataloader)
        earlystopping._stopping(val_loss)
        if earlystopping.stop:
            print('Early Stopping Excuated')
            break
        end = time.time()
        duration = (end - start) / 60
    
        print(f"Epoch: {epoch}, Test_Loss: {train_loss},Validation_loss: {val_loss}, Train_acc: {ep_tr_acc}, Val_acc: {ep_val_acc}")
    avg_train_acc.append(ep_tr_acc)
    avg_test_acc.append(ep_val_acc)
    avg_train_loss.append(train_loss.detach().item())
    avg_test_loss.append(float(val_loss.item()))
    print('Append list is printing')
    print(avg_train_acc,avg_test_acc,avg_test_loss,avg_train_loss)


    
# print(len(train_ids))
# print(len(test_ids))
    
# Val_dataloader = DataLoader(dataset = val_set1,  batch_size= 256, shuffle = False )
# validation_set1 = Val_dataloader.dataset

# len(validation_set1),len(val_set1),len(val_datasets)

********************* 0


/home/.conda/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/.conda/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 26/26 [10:07<00:00, 23.35s/it, tr_acc=866, train_loss=tensor(3.2730, grad_fn=<NllLossBackward0>)]


Epoch: 0, Test_Loss: 3.2730398178100586,Validation_loss: 1586.259765625, Train_acc: 0.06571060019728356, Val_acc: 0.03732362312243969


100%|██████████| 26/26 [09:57<00:00, 23.00s/it, tr_acc=2046, train_loss=tensor(2.7966, grad_fn=<NllLossBackward0>)]


Epoch: 1, Test_Loss: 2.796609878540039,Validation_loss: 3.363858699798584, Train_acc: 0.15524698383792396, Val_acc: 0.13154301319981793


...

Epoch: 7, Test_Loss: 0.6573319435119629,Validation_loss: 1.1200391054153442, Train_acc: 0.7827769347496206, Val_acc: 0.6628727521056226


100%|██████████| 26/26 [09:57<00:00, 22.98s/it, tr_acc=10765, train_loss=tensor(0.6459, grad_fn=<NllLossBackward0>)]


Epoch: 8, Test_Loss: 0.6458561420440674,Validation_loss: 2.2681260108947754, Train_acc: 0.8167678300455236, Val_acc: 0.5695424539039381


100%|██████████| 26/26 [09:57<00:00, 22.98s/it, tr_acc=11250, train_loss=tensor(0.3702, grad_fn=<NllLossBackward0>)]


Epoch: 9, Test_Loss: 0.3702341318130493,Validation_loss: 0.8347463607788086, Train_acc: 0.8535660091047041, Val_acc: 0.7587070339175962
Append list is printing
[0.8867137112072236, 0.8679817905918058, 0.8237481031866465, 0.8535660091047041] [0.8340919435593992, 0.7541543364443433, 0.6667425449578875, 0.7587070339175962] [0.4327267110347748, 0.8639750480651855, 1.144299864768982, 0.8347463607788086] [0.31219547986984253, 0.41286003589630127, 0.470316082239151, 0.3702341318130493]


In [ ]:
# mean = [0.4757, 0.5001, 0.4264]
# std = [0.2165, 0.1957, 0.2320]

# transforms = transform.Compose([transform.Resize((256,256)), transform.ToTensor(), transform.Normalize(mean = mean, std = std)])
# val_datasets = ImageFolder(root = train_data1, transform=transforms)
# train_data1 = int(0.80 * len(val_datasets))
# testing_data1 = len(val_datasets) - train_data1
# train_set1, val_set1 = torch.utils.data.random_split(val_datasets, [train_data1, testing_data1])
# kfold = KFold(n_splits=10)
# sec_avg_train_acc = []
# sec_avg_test_acc = []
# sec_avg_train_loss = []
# sec_avg_test_loss = []
# for fold, (train_ids, test_ids) in enumerate(kfold.split(train_set1)):
#     print('*********************',fold)
#     train_sampler = SubsetRandomSampler(train_ids)
#     test_sampler = SubsetRandomSampler(test_ids)
#     Train_dataloader = DataLoader(train_set1, batch_size = 256, sampler = train_sampler)
#     Val_dataloader = DataLoader(train_set1, batch_size = 256, sampler = test_sampler)
#     # from torchvision import models
#     # import torch.nn as nn
#     model = models.resnet18(pretrained=True)
# # model
# # Freeze all parameters in the pre-trained model
#     for param in model.parameters():
#         param.requires_grad = True
# # # Extracting features from last CNN layer
#     num_features = model.fc.in_features
#     model.fc = nn.Linear(num_features , 38)
#     # optimizer and loss function
#     # import torch.optim as optim
#     optimizer = optim.Adam(model.parameters(), lr = 1e-3)
#     loss_function = nn.CrossEntropyLoss()
#     earlystopping = EarlyStopping(patience=5)
#     for epoch in range(30):
#         start = time.time()
    
#         tr_acc = 0
#         val_acc = 0

    
#     # Train
#         model.train()
    
#         with tqdm(Train_dataloader, unit="batch") as tepoch:
#             for xtrain, ytrain in tepoch:
            
#                 train_prob = model(xtrain)
        
            
#                 train_loss = loss_function(train_prob, ytrain)
#                 train_loss.backward() # backpropagation

#             # loss.backward()
#                 optimizer.step() # update the model parameters
#                 optimizer.zero_grad()
#             # training ends
            
#                 train_pred = torch.max(train_prob, 1).indices
#                 tr_acc += int(torch.sum(train_pred == ytrain))
            
#             ep_tr_acc = tr_acc / len(train_ids)
#         end = time.time()
#         duration = (end - start) / 60
    
#     # print(f"Epoch: {epoch}, Test_Loss: {train_loss}, Train_acc: {ep_tr_acc}")
#     # Evaluate
    
#         model.eval()
# #     # with torch.no_grad():
#         with torch.no_grad():
#             for xval, yval in Val_dataloader:
#                 val_prob = model(xval)
            
#                 val_loss = loss_function(val_prob , yval)
            
#                 val_pred = torch.max(val_prob,1).indices
#                 val_acc += int(torch.sum(val_pred == yval))
            
#             ep_val_acc = val_acc / len(test_ids)
#         # avg_val_loss = val_loss / len(Val_dataloader)
#         earlystopping._stopping(val_loss)
#         if earlystopping.stop:
#             print('Early Stopping Excuated')
#             break
#         end = time.time()
#         duration = (end - start) / 60
    
#         print(f"Epoch: {epoch}, Test_Loss: {train_loss},Validation_loss: {val_loss}, Train_acc: {ep_tr_acc}, Val_acc: {ep_val_acc}")
#     sec_avg_train_acc.append(ep_tr_acc)
#     sec_avg_test_acc.append(ep_val_acc)
#     sec_avg_train_loss.append(train_loss.detach().item())
#     sec_avg_test_loss.append(float(val_loss.item()))
#     print('Append list is printing')
#     print(sec_avg_train_acc,sec_avg_test_acc,sec_avg_train_loss,sec_avg_test_loss)


    
# # print(len(train_ids))
# # print(len(test_ids))
    
# # Val_dataloader = DataLoader(dataset = val_set1,  batch_size= 256, shuffle = False )
# # validation_set1 = Val_dataloader.dataset

# # len(validation_set1),len(val_set1),len(val_datasets)

In [ ]:
# np.mean(avg_train_acc), np.mean(avg_test_acc), np.mean(avg_test_loss), np.mean(avg_train_loss)

In [ ]:

# dataset = torch.tensor([ 9, 8,7,6,5,4,3,2,1])
# # Define the indices of the subset you want to sample from the dataset
# # train_ids = [0, 1, 2]  # Example subset indices
# kfold = KFold(n_splits=3)
# for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
#     # Print
#     print(f'FOLD {fold}')
#     # print(
# # Create a SubsetRandomSampler using the train_ids
#     sampler = SubsetRandomSampler(train_ids)
#     test_sampler       =  SubsetRandomSampler(test_ids)
# # Create a DataLoader using the sampler
#     dataloader = DataLoader(dataset, batch_size=32,sampler=sampler)
#     for batch in dataloader:
#         print(batch)
#     dataloader = DataLoader(dataset, batch_size=32, sampler=test_sampler)
#     for batch in dataloader:
#         print(batch)